In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
from tqdm.notebook import tqdm

import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.metrics import mean_squared_error


In [ ]:
training_file = pd.read_csv("../input/commonlitreadabilityprize/train.csv")
test_file = pd.read_csv("../input/commonlitreadabilityprize/test.csv")
submission = pd.read_csv("../input/commonlitreadabilityprize/sample_submission.csv")


In [ ]:
training_file.head()

In [ ]:
training_file.shape

In [ ]:
data = training_file[['excerpt', 'target']]
data = data.sample(frac=1).reset_index(drop=True)
excerpt, targets = training_file['excerpt'].values, training_file['target'].values

t_X, v_X = excerpt[:2750], excerpt[2750:]
t_Y, v_Y = targets[:2750], targets[2750:]

print(t_X.shape, v_X.shape)
print(t_Y.shape, v_Y.shape)


In [ ]:
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor

In [ ]:
models={"LR":LinearRegression(),
        "rdg":Ridge(),
        "rf":RandomForestRegressor(n_estimators=100),
        "Adb":AdaBoostRegressor(n_estimators=120)
        }

In [ ]:
def train(model,vec):
    model_pipeline=make_pipeline(
    vec,
    model
    )
    model_pipeline.fit(t_X, t_Y)

    preds = model_pipeline.predict(v_X)
    mse_loss = mean_squared_error(v_Y, preds)

    print(f"MSE Loss using {name} model is: {mse_loss}")


<h2>Using Countvectorizer

In [ ]:
for name,model in zip(models.keys(),models.values()):
        train(model,vec=CountVectorizer(binary=True))

<h2>Using TfidfVectorizer 

In [ ]:
for name,model in zip(models.keys(),models.values()):
        train(model, vec=TfidfVectorizer(binary=True, ngram_range=(1, 1)))

In [ ]:
#From above we can see that ridge regression has minimum loss
final_model=make_pipeline(
                TfidfVectorizer(binary=True, ngram_range=(1, 1)),
                models['rdg'])
final_model.fit(t_X, t_Y)

In [ ]:
test = test_file[['id', 'excerpt']]
test_ids = test['id'].tolist()
test_text = test['excerpt'].values

test_preds_linear = final_model.predict(test_text)

submission = pd.DataFrame()
submission['id'] = test_ids
submission['target']=test_preds_linear
submission.to_csv("submission.csv", index=None)